The following notebook computes the quantitative assessment metrics of the single and multi-visit reconstruction produced in Steps 00-03. The metrics are computed on a third independent test set which does not share any scans from subjects within the train and validation sets. The test set consists of 7 longitudinal pairs of scans. The SSIM and pSNR of the single-visit reconstructions are compared to the multi-visit reconstructions to measure the improvements when incorporating previous subject-specific information. The first and last 20 slices are removed before computing metrics to eliminate slices with little to no anatomical structures. 

In [2]:
import glob
import nibabel
import sys
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
MY_UTILS_PATH = "../src/"
if not MY_UTILS_PATH in sys.path:
    sys.path.append(MY_UTILS_PATH)
import metrics

In [3]:
test_files = np.loadtxt('../data/train_val_test_split/test_long.txt',dtype=str)
print(test_files)

[['TUM04-20171108.nii' 'TUM04-20190328.nii']
 ['TUM04-20180618.nii' 'TUM04-20181009.nii']
 ['TUM20-20180205.nii' 'TUM20-20180402.nii']
 ['TUM10-20171018.nii' 'TUM10-20180122.nii']
 ['TUM10-20171018.nii' 'TUM10-20180307.nii']
 ['TUM15-20170531.nii' 'TUM15-20170801.nii']
 ['TUM15-20170801.nii' 'TUM15-20170816.nii']]


In [4]:
#file pathes
initial_path = '../data/predicted/10x-iki/'
previous_path = '../data/reference_reg_10x-iki/'
enhanced_path = '../data/predicted/10x-enhanced-iki/'
ref_path = '../../../data/brain-cancer/'

In [5]:
#file ids with pathes
initial_files = [initial_path + file[:-4] + '_predicted.nii' for file in test_files[:,1]]
previous_files = [previous_path + 'elastic_' + file[0][:-4] + '_' + file[1] for file in test_files]
enhanced_files = [enhanced_path + file[:-4] + '_predicted.nii' for file in test_files[:,1]]
ref_files = [ref_path + file for file in test_files[:,1]]

In [6]:
init_mets = []
prev_mets = []
enh_mets = []
for ii in range(len(test_files)):
    init = nib.load(initial_files[ii]).get_fdata()
    prev = nib.load(previous_files[ii]).get_fdata()
    enh = nib.load(enhanced_files[ii]).get_fdata()
    ref = nib.load(ref_files[ii]).get_fdata()
    
    init = np.swapaxes(init,0,2)
    prev = np.swapaxes(prev,0,2)
    enh = np.swapaxes(enh,0,2)
    ref = np.swapaxes(ref,0,2)
    
    init = init / np.abs(init).max()
    prev = prev / np.abs(prev).max()
    enh = enh / np.abs(enh).max()
    ref = ref / np.abs(ref).max()
    
    init_mets.append(metrics.metrics(init,ref))
    prev_mets.append(metrics.metrics(prev,ref))
    enh_mets.append(metrics.metrics(enh,ref))
    
    

In [7]:
print(init.shape, prev.shape, enh.shape, ref.shape)

(165, 512, 512) (165, 512, 512) (165, 512, 512) (165, 512, 512)


In [12]:
init_cropped = []
enh_cropped = []
for ii in range(len(init_mets)):
    init_cropped.append(init_mets[ii][0][20:-20])
    enh_cropped.append(enh_mets[ii][0][20:-20])
#    print(enh_mets[ii][0][20:-20].mean())

In [13]:
init_cropped = np.concatenate(init_cropped,axis=0)
enh_cropped = np.concatenate(enh_cropped,axis=0)

In [14]:
print(init_cropped.mean(), enh_cropped.mean())

0.8649802402326393 0.9515280475115492
